# Creating Agents to generate recipes based on user's ingredients.

Here, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.



In [ ]:
from google.colab import userdata
# together_api = userdata.get('TOGETHER_API')
openai_api = userdata.get('OPENAI_API')

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install crewai crewai-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.0/192.0 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.0/468.0 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
# from together import Together
from crewai import Agent, Task, Crew, LLM
import openai

os.environ["OPENAI_API_KEY"] = openai_api

In [ ]:
class OpenAILLM:
    def __init__(self, model_name="gpt-3.5-turbo"):
        self.model_name = model_name

    def generate(self, prompt):
        # Use OpenAI's API to generate a response based on the prompt
        response = openai.ChatCompletion.create(
            model=self.model_name,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=500,  # You can adjust this
            temperature=0.7  # Adjust temperature for creativity
        )
        return response.choices[0].message["content"].strip()
llm = OpenAILLM()

In [ ]:
# llm = LLM(
#     model="Llama-3.2-11B-Vision-Instruct-Turbo",
#     temperature=0.7,
#     base_url="https://api.together.ai/models/meta-llama/Llama-3.2-11B-Vision-Instruct-Turbo",
#     api_key=together_api
# )

# llm = LLM(
#     model="fireworks_ai/accounts/fireworks/models/llama-v3p2-3b",
#     temperature=0.7
# )


### Creating Agents

In [ ]:
## researcher
recipe_researcher = Agent(
    role = "Recipe Researcher",
    goal= "Research these ingredients: {ingredients}, and generate creative recipes",
    backstory = "You're a seasoned chef with an extensive knowledge of ingredients"
                "and how they come together in recipes. You always find new ways to combine flavors and create unique dishes.",
    verbose = True,
    allow_delegation = True,
    llm = llm
)

In [ ]:
## developer
recipe_developer = Agent(
    role = "Recipe Developer",
    goal= "Develop a recipe using these ingredients: {ingredients}",
    backstory = "You're a creative cook with a knack for developing new recipes."
                " You love experimenting with different ingredients and flavors to create unique dishes.",
    verbose = True,
    llm = llm
)

### Creating Tasks

In [ ]:
##  research
research = Task(
    description=(
        "1. Research the provided ingredients to understand their flavor profiles, nutritional benefits, "
        "and common usage in recipes.\n"
        "2. Investigate current recipe trends and creative ways to combine these ingredients.\n"
        "3. Generate a list of recipe ideas using these ingredients, considering various cooking methods and cuisines.\n"
        "4. Suggest ingredient variations and substitutes based on research.\n"
        "5. Include any interesting food pairings or techniques that enhance the recipe."
    ),
    expected_output="A list of creative recipe ideas based on the ingredients, "
        "including preparation methods, ingredient variations, and food pairing suggestions.",
    agent=recipe_researcher,
)

In [ ]:
## chef
make = Task(
    description=(
        "1. Create a recipe using the provided ingredients.\n"
        "2. Develop a detailed recipe with step-by-step instructions, including ingredient quantities, "
        "cooking methods, and serving suggestions.\n"
        "3. Consider the flavor profiles of the ingredients and how they complement each other in the dish.\n"
        "4. Include any variations or substitutions for the ingredients based on availability or dietary restrictions."

    ),
    expected_output="A complete recipe with detailed instructions, ingredient quantities, "
        "cooking methods, and serving suggestions.",
    output_file="recipe.md",
    agent=recipe_developer,

)

### Setting up the crew

In [ ]:
crew = Crew(
    agents=[recipe_researcher, recipe_developer],
    tasks=[research, make],
    verbose=True
)

### Output

In [ ]:
result = crew.kickoff(inputs={"ingredients": "chicken, broccoli, lemon, honey, garlic"})

# Agent: Recipe Researcher
## Task: 1. Research the provided ingredients to understand their flavor profiles, nutritional benefits, and common usage in recipes.
2. Investigate current recipe trends and creative ways to combine these ingredients.
3. Generate a list of recipe ideas using these ingredients, considering various cooking methods and cuisines.
4. Suggest ingredient variations and substitutes based on research.
5. Include any interesting food pairings or techniques that enhance the recipe.
# Agent: Recipe Developer
## Task: Research the flavor profiles, nutritional benefits, and common usage of the ingredients: chicken, broccoli, lemon, honey, garlic


# Agent: Recipe Developer
## Final Answer: 
- **Chicken**:
   Flavor profile: Chicken has a mild, versatile flavor that pairs well with a variety of seasonings and ingredients. It can be grilled, baked, sautéed, or roasted to create different textures and flavors.
   Nutritional benefits: Chicken is a good source of lean protein

In [ ]:
print(result)


# Honey-Garlic Lemon Chicken and Broccoli Stir-Fry

## Ingredients:
- 1 lb chicken breast, cut into strips
- 2 cups broccoli florets
- 3 cloves garlic, minced
- 1/4 cup honey
- 2 tablespoons soy sauce
- Juice of 1 lemon
- 2 tablespoons olive oil
- Salt and pepper to taste
- Cooked rice or noodles, for serving
- Sesame seeds and sliced green onions for garnish

## Instructions:

1. **Marinate the Chicken**:
   - In a bowl, combine minced garlic, honey, soy sauce, lemon juice, salt, and pepper.
   - Add the chicken strips to the marinade, making sure they are well coated. Let it marinate for at least 30 minutes in the refrigerator.

2. **Stir-Fry**:
   - Heat olive oil in a large pan over medium-high heat.
   - Add marinated chicken strips to the pan and cook until they are browned and cooked through.
   - Add broccoli florets to the pan and stir-fry for an additional 3-4 minutes or until they are tender-crisp.
  
3. **Combine**:
   - Once the chicken and broccoli are cooked, pour any re